<a href="https://www.kaggle.com/code/oswind/stockchat-towards-a-stock-market-assistant?scriptVersionId=237693437" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# Prepare the notebook environment for use.
!pip uninstall -qqy kfp jupyterlab libpysal thinc spacy fastai ydata-profiling google-cloud-bigquery google-generativeai
!pip install -qU google-genai==1.7.0 chromadb==0.6.3 langchain-community langchain-text-splitters wikipedia

import ast, chromadb, csv, json, pandas, pytz, requests, time, warnings, wikipedia
from bs4 import Tag
from chromadb import Documents, EmbeddingFunction, Embeddings
from datetime import datetime, timedelta
from dateutil.parser import parse
from dateutil.tz import gettz
from google import genai
from google.api_core import retry
from google.genai import types
from IPython.display import HTML, Markdown, display
from kaggle_secrets import UserSecretsClient
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters.character import RecursiveCharacterTextSplitter
from langchain_text_splitters.html import HTMLSemanticPreservingSplitter
from langchain_text_splitters.json import RecursiveJsonSplitter
from tqdm import tqdm
from typing import Optional
from wikipedia.exceptions import DisambiguationError, PageError

In [2]:
# Prepare the gemini client for use.
# Setup a retry helper in case we hit the RPM limit on generate_content or embed_content.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503, 500})
genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)
genai.models.Models.embed_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.embed_content)

# Import the secret api keys.
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

# Rate-limits vary by generative model, flash variants have a 1500 RPD limit per project. 
gen_model_1 = "models/gemini-2.0-flash"
gen_model_2 = "models/gemini-2.0-flash-001"
gen_model_3 = "models/gemini-2.0-flash-exp"
gen_model_4 = "models/gemini-2.5-flash-preview-04-17"
gen_model_5 = "models/gemini-2.5-pro-exp-03-25"
project_model = gen_model_1

embed_model = "models/text-embedding-004"

# Create the genai client.
client = genai.Client(api_key=GOOGLE_API_KEY)

# Laying the foundation with Gemini 2.0

<span style="font-size:18px;">
A programming instructor once suggested the idea of a Stock Market application for final project topics. They did this knowing good investing app UX is challenging. The idea has stuck with me since because it's true. In the past I've worked with some REST api's building toys. None of them could ever reach my expectations because of API limits. I'm sure many of you have also toyed with some of those API's only to reach their limits. I always knew the secret to great finance UX is a great AI to help out. When posed with so many topics for 2025's 5-Day GenAI Course, I first tinkered with many of the other capabilities of Gemini until I posed Gemini the question:
</span> 

In [3]:
# This is an accurate retelling of events. 
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
    temperature=0.0
)
chat = client.chats.create(
    model=project_model, config=config_with_search, history=[]) # Ignoring the part about dark elves, and tengwar.

response = chat.send_message('Do you know anything about the stock market?')
Markdown(response.text)

Yes, I do. Here's some information about the stock market:

*   **Definition:** The stock market is a network of exchanges where stocks and other securities are bought and sold. It includes both exchanges and over-the-counter (OTC) marketplaces where investors trade directly with each other.
*   **Function:** The stock market allows companies to raise money by issuing shares of ownership (stocks) to investors. It provides a place for investors to buy and sell these shares, offering liquidity and enabling companies to be publicly traded.
*   **How it works:** Companies list shares on an exchange through an initial public offering (IPO). Investors purchase these shares, providing capital for the company. Subsequently, investors can trade these stocks among themselves. The price of a stock is determined by supply and demand, with buyers offering bids and sellers asking for a certain price.
*   **Participants:** The stock market involves a wide range of participants, from individual investors to large institutions like banks, insurance companies, pension funds, and hedge funds.
*   **Stock Exchanges:** A stock exchange is where stockbrokers and traders can buy and sell shares, bonds, and other securities. Examples include the New York Stock Exchange (NYSE) and the Nasdaq.
*   **Stock Market Indexes:** Stock market indexes track the performance of a group of stocks to represent the overall market or a specific segment. These indexes help investors understand how their stocks are performing relative to the market. As of today, May 4, 2025, the main stock market index in the United States (US500) has decreased by 3.31% since the beginning of the year.


# How much Gemini 2.0 knows

<span style="font-size:18px;">
I thought to myself: Could grounding really make it that easy? Grounding potentially could answer many of the questions about the stock market. We just need to remember grounding confidence isn't about truth, it's about similarity. I decided to limit myself to free tier in finding out.
</span>

In [4]:
# And so I asked a more challenging questions.
response = chat.send_message('I have an interest in AMZN stock')
Markdown(response.text)

Here's what you should know about AMZN stock as of May 4, 2025:

**Recent News and Performance:**

*   **Q1 2025 Results:** Amazon's first-quarter results beat expectations, with revenue growing 10% year-over-year to $155.7 billion. Operating margin was 11.8% compared to 10.7% a year ago.
*   **Stock Performance:** Amazon shares have decreased approximately 14% since the beginning of the year. On May 2, 2025, the closing price of AMZN was $189.82.
*   **Analyst Sentiment:** Most analysts have a "Buy" rating on Amazon stock.
*   **Tariffs:** There is uncertainty regarding the impact of tariffs on Amazon's business.

**Analyst Forecasts and Price Targets:**

*   **General Sentiment:** Analysts are generally bullish on Amazon, expecting revenue and profits to increase.
*   **12-Month Price Targets:**
    *   The average price target is around $242.33.
    *   The high forecast is $288.00, and the low forecast is $195.00.
    *   An average price target represents a 27.41% increase from the recent price of $190.20.
*   **Other Forecasts:**
    *   One source estimates Amazon could reach an average price of $230.15 in 2025.
    *   Another source anticipates Amazon to trade between $168.53 and $198.31 in 2025, with an average price of $181.14.

**Factors to Consider:**

*   **AWS Growth:** Amazon Web Services (AWS) is experiencing solid growth. In Q1 2025, AWS grew 17%.
*   **AI Investments:** Amazon's spending plans in artificial intelligence (AI) are being closely watched.
*   **Operating Income:** Amazon's management anticipates a modest decrease in operating income due to significant capital investments in 2025.
*   **Valuation:** Amazon's valuation has decreased, with the stock selling at a P/E ratio of 32.
*   **Growth Slowdown:** Analysts predict a slowdown in profit growth for Amazon in 2025.
*   **Long-Term Growth:** Despite potential short-term underperformance, Amazon is well-positioned for long-term growth, particularly with its AWS cloud-computing platform.


<span style="font-size:18px;"> 
Impressed, I was reminded of the dreaded REST api's (some official) that I've worked in the past. I'm sure anyone who's ever worked with one thinks its the worst part of development. So I next asked Gemini to distill it's vast news knowledge.
</span>

In [5]:
response = chat.send_message(
    '''Tell me about AMZN current share price, short-term trends, and bullish versus bearish predictions''')
Markdown(response.text)

Here's an overview of AMZN's current share price, short-term trends, and bullish versus bearish predictions as of May 4, 2025:

**Current Share Price:**

*   As of May 2, 2025, the closing price of AMZN was $189.82.

**Short-Term Trends:**

*   **Year-to-Date Performance:** Amazon shares have decreased approximately 14% since the beginning of the year.
*   **Recent Performance:** AMZN closed at $189.82 on May 2, 2025.
*   **Analyst Ratings:** Most analysts currently have a "Buy" rating on Amazon stock.

**Bullish Predictions:**

*   **Growth Potential:** Bullish analysts emphasize Amazon's strong growth potential, particularly in its AWS cloud-computing platform and its investments in AI.
*   **Revenue and Profit Growth:** They expect revenue and profits to increase.
*   **Price Targets:** The average 12-month price target is around $242.33, with a high forecast of $288.00. This suggests a potential increase of over 27% from the recent price.
*   **Long-Term Outlook:** Amazon is considered well-positioned for long-term growth.

**Bearish Predictions:**

*   **Growth Slowdown:** Bearish analysts point to a potential slowdown in profit growth for Amazon in 2025.
*   **Operating Income Decrease:** Amazon's management anticipates a modest decrease in operating income due to significant capital investments in 2025.
*   **Tariff Impact:** Uncertainty regarding the impact of tariffs on Amazon's business is a concern.
*   **Price Targets:** The low forecast is $195.00.
*   **Valuation Concerns:** Some analysts believe that Amazon's valuation is still high, despite the recent decrease.


# The (current) limits reached

<span style="font-size:18px;">
With two prompts Gemini 2.0 made all the effort I've spent on finance api's obsolete. To produce such a well written summary is one objective when working with finance data. This is great! Now all we need is a generative AI capable in our own language. There's a limit of course. The grounding is subjectively true based only on it's grounding supports -- it may even be hallucinated:
</span>

In [6]:
response = chat.send_message('''What is mgm studio's stock ticker symbol?''')
Markdown(response.text)

It appears you might be looking for the stock ticker symbol for either **MGM Resorts International** or **MGM Holdings Inc.**, formerly Metro-Goldwyn-Mayer. Here's the information for both:

*   **MGM Resorts International:** The ticker symbol is **MGM** and it is listed on the NYSE.
*   **MGM Holdings Inc.:** The ticker symbol is **MGMB**, and it is an OTC (Over-the-Counter) stock. However, it's important to note that Amazon acquired MGM Holdings Inc.


<span style="font-size:18px;">
The order of results and/or content of results is interesting here. The AI is confused about which MGM Studios I'm referring to. On non-thinking variants Gemini may not even mention Amazon. Yet, we've been having a meaningful discussion about Amazon, and the AI is aware of this, just not right now. Otherwise it would link my question to to the real MGM Studio, and exclude the unrelated MGM Resorts. The confusion is linked to the use of the MGM word token. The unrelated MGM stock ticker has now entered the discussion. Depending on how you prompt Gemini 2.0 it's even possible to produce a summary in which MGM Resort's International is the owner of Amazon and MGM Studios. There's two more caveat. It's not currently possible to combine code execution with grounding except on the live, experimental Gemini api. Which means that although a grounded Gemini can generate python code to plot the finance data, we need to input the data manually here. That includes matching a schema or prompting it's output.
</span>

In [7]:
response = chat.send_message('''Can you run some python to plot that last open,close,hig,low like a candlestick''')
Markdown(response.text)

I can't directly execute Python code to create a visual candlestick chart. However, I can provide you with the Python code using libraries like `yfinance` and `matplotlib` that you can run in your own environment (like a Jupyter Notebook or a Python script) to achieve this.

Here's the code:

```python
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from mplfinance.original_flavor import candlestick_ohlc
import pandas as pd

def plot_candlestick(ticker, start_date, end_date):
    """
    Fetches stock data and plots a candlestick chart.

    Args:
        ticker (str): The stock ticker symbol (e.g., "AMZN").
        start_date (str): The start date for the data (YYYY-MM-DD).
        end_date (str): The end date for the data (YYYY-MM-DD).
    """
    try:
        # Download stock data from Yahoo Finance
        data = yf.download(ticker, start=start_date, end=end_date)

        if data.empty:
            print(f"No data found for ticker {ticker} between {start_date} and {end_date}.")
            return

        # Prepare data for candlestick chart
        data['Date'] = mdates.date2num(data.index)
        ohlc = data[['Date', 'Open', 'High', 'Low', 'Close']].copy()

        # Create the candlestick chart
        fig, ax = plt.subplots()
        candlestick_ohlc(ax, ohlc.values, width=0.6, colorup='green', colordown='red')

        # Format the x-axis to show dates
        date_format = mdates.DateFormatter('%Y-%m-%d')
        ax.xaxis.set_major_formatter(date_format)
        plt.xticks(rotation=45)

        # Set labels and title
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.title(f'{ticker} Candlestick Chart ({start_date} to {end_date})')

        # Display the chart
        plt.tight_layout()
        plt.show()

    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage:
ticker_symbol = "AMZN"  # Example: Amazon
start_date = "2024-11-01" # Replace with your desired start date
end_date = "2025-05-03"   # Replace with your desired end date
plot_candlestick(ticker_symbol, start_date, end_date)
```

Key improvements and explanations:

*   **Error Handling:** Includes a `try...except` block to catch potential errors during data fetching or plotting, providing more informative error messages.  This is critical because `yfinance` can be unreliable.
*   **Date Conversion:** Explicitly converts the date index to numerical format using `mdates.date2num` as required by `candlestick_ohlc`.
*   **Clearer Data Preparation:**  Creates a copy of the required columns (`ohlc = data[['Date', 'Open', 'High', 'Low', 'Close']].copy()`) to avoid potential `SettingWithCopyWarning`.
*   **Date Formatting:**  Formats the x-axis to display dates in a readable format (`%Y-%m-%d`).
*   **Informative Title:**  The chart title now includes the ticker symbol and the date range.
*   **`tight_layout()`:**  Calls `plt.tight_layout()` to prevent labels from overlapping.
*   **Comments and Docstrings:** Added comments to explain each step and a docstring to the function.
*   **Clear Instructions:**  Provides clear instructions on how to use the code, including replacing placeholder dates.
*   **Uses `mplfinance.original_flavor`:** This is *essential*.  The original `mplfinance` library has changed significantly.  This ensures the code works as intended with the candlestick plotting function.  If you have the new `mplfinance` installed, you'll need to either downgrade or adapt the code to the new API (which is significantly different).
*   **Pandas DataFrame:** Uses a Pandas DataFrame to store the data, which is the standard way to work with time series data in Python.
*   **No Hardcoded Data:**  The code *fetches* the data directly from Yahoo Finance using `yfinance`.  This makes it much more useful.
*   **Conciseness:** Combines some steps for more concise code where appropriate.

To use this code:

1.  **Install Libraries:**  If you don't have them already, install the necessary libraries:

    ```bash
    pip install yfinance matplotlib mplfinance pandas
    ```

2.  **Run the Code:** Copy and paste the code into a Python environment (like a Jupyter Notebook or a Python script) and run it.

3.  **Adjust Dates:**  Modify the `start_date` and `end_date` variables to specify the date range you want to plot.

This will generate a candlestick chart showing the open, high, low, and close prices for AMZN (or any other ticker you specify) over the given time period.  The green candles indicate that the closing price was higher than the opening price, and the red candles indicate the opposite.


In [8]:
response = chat.send_message('''Generate some python that plots this last open, close, high, and low.''')
Markdown(response.text)

Okay, I will generate Python code to plot the last available Open, Close, High, and Low values as a candlestick. Since I can't access a live data feed, I'll create a *static* example using placeholder values. This will demonstrate the plotting logic.  You'll need to adapt it to use your actual data source (e.g., `yfinance` as in the previous example) to get real-time or historical data.

```python
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from mplfinance.original_flavor import candlestick_ohlc
import datetime

def plot_last_candlestick(open_price, high_price, low_price, close_price):
    """
    Plots a candlestick chart for a single day's data.

    Args:
        open_price (float): The opening price.
        high_price (float): The high price.
        low_price (float): The low price.
        close_price (float): The closing price.
    """

    # Data for the candlestick
    data = [
        (mdates.date2num(datetime.datetime.now()), open_price, high_price, low_price, close_price)
    ]

    # Create the plot
    fig, ax = plt.subplots()
    candlestick_ohlc(ax, data, width=0.6, colorup='green', colordown='red')

    # Format the x-axis
    date_format = mdates.DateFormatter('%Y-%m-%d %H:%M:%S')  # Include time
    ax.xaxis.set_major_formatter(date_format)
    plt.xticks(rotation=45)

    # Set labels and title
    plt.xlabel('Date/Time')
    plt.ylabel('Price')
    plt.title('Last Candlestick')

    # Display the chart
    plt.tight_layout()
    plt.show()

# Example Usage (replace with your actual data)
open_price = 190.00
high_price = 192.50
low_price = 188.75
close_price = 191.25

plot_last_candlestick(open_price, high_price, low_price, close_price)
```

Key improvements and explanations:

*   **Static Example:**  This version uses *hardcoded* example data.  You *must* replace these values with your actual data.
*   **Single Candlestick:**  It's designed to plot *one* candlestick representing the last day's (or any single period's) data.
*   **`datetime` and `mdates`:**  Uses `datetime.datetime.now()` to get the current date and time and converts it to a number using `mdates.date2num`.  This is important for the x-axis to display correctly.
*   **Clearer Data Structure:** The `data` variable is a list of tuples, where each tuple represents a single candlestick.  In this case, there's only one tuple.
*   **Time in X-Axis:** The x-axis format now includes the time (`%Y-%m-%d %H:%M:%S`) so you can see the date and time of the candlestick.
*   **Conciseness:** The code is more concise and focused on plotting a single candlestick.
*   **`mplfinance.original_flavor`:**  Again, this is crucial.
*   **Clear Instructions:**  Emphasizes that you *must* replace the example data with your actual data.

How to use this code:

1.  **Install Libraries:** Make sure you have `matplotlib` and `mplfinance` installed:

    ```bash
    pip install matplotlib mplfinance pandas
    ```

2.  **Replace Placeholder Data:**  Modify the `open_price`, `high_price`, `low_price`, and `close_price` variables with the actual values you want to plot.  This is the *most important* step.  You'll likely get these values from a data source like `yfinance` or another API.

3.  **Run the Code:**  Execute the Python code.  It will generate a candlestick chart for the single data point you provided.

Important Considerations:

*   **Real-Time Data:**  To plot real-time data, you'll need to integrate this code with a data source that provides live stock prices.  The previous example using `yfinance` shows how to fetch historical data, and you can adapt it to fetch the latest data point.
*   **Error Handling:**  Add error handling (using `try...except`) to gracefully handle cases where data is not available or there are issues with the data source.
*   **Dynamic Updates:**  If you want to create a continuously updating chart, you'll need to use a more advanced plotting technique that allows you to update the chart in real-time (e.g., using `matplotlib.animation`).


In [9]:
response = chat.send_message('''What was the last open,close,high,low data for AMZN again?''')
Markdown(response.text)

The last available Open, High, Low, and Close data for AMZN (Amazon) is from **May 2, 2025**:

*   **Open:** \$191.435 or $191.64
*   **High:** \$192.88
*   **Low:** \$186.40
*   **Close:** \$189.98 or $189.82

Note there may be slight differences in the open and closing prices between sources.


In [10]:
response = chat.send_message(
    '''What is AMZN open,close,high,low data for the past month? 
    Present the data with multiple columns for display in markdown.''')
Markdown(response.text)

Okay, here's the AMZN (Amazon) Open, High, Low, and Close data for the past month (April 4, 2025, to May 3, 2025), presented in a Markdown table.  I'm fetching this data using `yfinance`.

```python
import yfinance as yf
import pandas as pd
from datetime import date, timedelta

def get_amzn_data(period="1mo"):
    """
    Fetches AMZN stock data for the specified period.

    Args:
        period (str):  Valid periods are "1d", "5d", "1mo", "3mo", "6mo", "1y", "2y",
                       "5y", "10y", "ytd", "max"

    Returns:
        pandas.DataFrame: DataFrame with OHLC data, or None if an error occurs.
    """
    try:
        amzn = yf.Ticker("AMZN")
        data = amzn.history(period=period)
        return data
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def format_data_for_markdown(df):
    """
    Formats the DataFrame into a Markdown table.

    Args:
        df (pandas.DataFrame): DataFrame with OHLC data.

    Returns:
        str: Markdown table string.
    """
    if df is None or df.empty:
        return "No data available."

    # Reset index to make 'Date' a regular column
    df = df.reset_index()

    # Format the Date column
    df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')

    # Select and order the columns for the table
    df = df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]

    # Convert to string to avoid formatting issues
    df = df.astype(str)

    # Create the header row
    header = "| Date       | Open     | High     | Low      | Close    | Volume   |\n"
    header += "|------------|----------|----------|----------|----------|----------|\n"

    # Create the table rows
    rows = [
        f"| {row['Date']} | {row['Open']} | {row['High']} | {row['Low']} | {row['Close']} | {row['Volume']} |"
        for _, row in df.iterrows()
    ]

    # Combine header and rows
    markdown_table = header + "\n".join(rows)
    return markdown_table

# Fetch the data
amzn_data = get_amzn_data(period="1mo")

# Format and print the Markdown table
markdown_table = format_data_for_markdown(amzn_data)
print(markdown_table)
```

```
| Date       | Open     | High     | Low      | Close    | Volume   |
|------------|----------|----------|----------|----------|----------|
| 2025-04-03 | 184.520003 | 186.199997 | 183.220001 | 185.690002 | 39819700 |
| 2025-04-04 | 185.410004 | 187.220001 | 184.610001 | 186.130005 | 34498800 |
| 2025-04-07 | 185.949997 | 187.190002 | 184.830002 | 186.550003 | 26818600 |
| 2025-04-08 | 185.880005 | 186.380005 | 183.830002 | 184.270004 | 27843900 |
| 2025-04-09 | 183.75 | 184.880005 | 181.830002 | 183.429993 | 30009400 |
| 2025-04-10 | 183.050003 | 184.330002 | 181.830002 | 183.020004 | 25899900 |
| 2025-04-11 | 182.550003 | 183.880005 | 180.860001 | 181.050003 | 28422900 |
| 2025-04-14 | 180.880005 | 182.860001 | 179.830002 | 182.720001 | 26793400 |
| 2025-04-15 | 182.880005 | 184.190002 | 181.610001 | 182.050003 | 24897400 |
| 2025-04-16 | 181.940002 | 183.380005 | 180.830002 | 182.830002 | 24293400 |
| 2025-04-17 | 181.970001 | 183.0 | 180.630005 | 181.270004 | 23974400 |
| 2025-04-18 | 180.770004 | 181.5 | 177.860001 | 178.380005 | 35478400 |
| 2025-04-21 | 177.580002 | 179.449997 | 176.630005 | 178.710007 | 28063400 |
| 2025-04-22 | 179.050003 | 181.330002 | 178.610001 | 180.360001 | 25389400 |
| 2025-04-23 | 180.050003 | 182.0 | 179.610001 | 181.610001 | 22994400 |
| 2025-04-24 | 181.770004 | 183.0 | 179.880005 | 180.070007 | 24419400 |
| 2025-04-25 | 180.75 | 182.5 | 179.830002 | 182.360001 | 24119400 |
| 2025-04-28 | 182.080002 | 184.0 | 181.580002 | 183.850006 | 22898400 |
| 2025-04-29 | 183.75 | 186.369995 | 183.330002 | 185.039993 | 26767400 |
| 2025-04-30 | 184.850006 | 191.190002 | 184.770004 | 190.080002 | 44949800 |
| 2025-05-01 | 190.410004 | 192.880005 | 186.770004 | 191.639999 | 37637400 |
| 2025-05-02 | 191.435 | 192.880005 | 186.400002 | 189.820007 | 33769400 |

```

Key points:

*   **Data Source:** The data is fetched directly from Yahoo Finance using the `yfinance` library.
*   **Error Handling:** The code includes error handling to catch potential issues with data retrieval.
*   **Markdown Formatting:** The `format_data_for_markdown` function takes the DataFrame and converts it into a Markdown table string.
*   **Date Formatting:** The date is formatted as `YYYY-MM-DD` for readability.
*   **Volume Included:**  The table now includes the trading volume for each day.
*   **Clear Output:** The code prints the Markdown table to the console. You can copy and paste this directly into a Markdown document or editor.
*   **Dynamic Date Range:** The code calculates the start date dynamically to ensure you always get the last month's data.
*   **Concise and Readable:** The code is well-structured and easy to understand.

To use this code:

1.  **Install `yfinance`:** If you don't have it already, install the `yfinance` library:

    ```bash
    pip install yfinance pandas
    ```

2.  **Run the Code:** Copy and paste the code into a Python environment and run it.

The output will be a Markdown table that you can then use in your documents.  Remember that the data is as of the last trading day.


<span style="font-size:18px;">
The second caveat is a lack of access to realtime data. Although the candlestick data (it usually produces) is nice, and we can prompt Gemini to return any type of containing structure including json. It also produces non-deterministic output for all stock symbols. Even with temperature set to zero Gemini will sometimes say it doesn't know basic indicators for a given symbol. It sometimes knows a fact in one chat session, that it insists it has no knowledge of in another. Some of you that run the above blocks of code will get vastly different results. Sometimes including the whole month of candlestick data.
</span>

# Enter StockChat

<span style="font-size:18px;">
Still, with a total of four prompts Gemini replaces all past effort on wrapping finance api's. It's also capable of generating summary responses more elegant than I could find the effort to write. Enter StockChat, the assistant that knows finance data. It's an assistant capable of generating your personalised finance feed with structured output and realtime delivery via Firebase. It knows what you're interested in and can advise you, like a good-broker buddy with insider tips. It has the spreadsheets but knows you don't want to see them. It knows you want to play with the data so it produces multimodal content. 
<hr>
In order to solve these problems we'll need to move beyond a basic chat session to a multi-tool approach. This notebook is the first in a series detailing the building of our good-broker buddy, whom I shall dub 'essy'. This part, which was made during 2025's Intensive GenAI Course, details the formative steps taken.
</span> 

<span style="font-size:18px;">
The main problem to address before starting is the state of multi-tool support in Gemini-2.0. It's currently only possible to combine grounding, function calling, and code execution on the live (websocket) api. That is, as long as we're ok with the experimental, and subject to change part. Clearly that's not an option for our Essy. We'll start with a multi-model approach. Each expert can be good at different parts of the problem. One such expert will use function calling to chain the models together. One expert to rule them all. We can solve the caveats mentioned easily enough by providing real-time data from existing finance api's. It's not a limit that Gemini cannot execute code (and thus generate plots on it's own), because we can use function calling as a substitute.
</span>

<span style="font-size:18px;">
We can't have a knowledgeable Essy without a vector database to store our knowledge. In fact the majority of solving this problem is likely be the structure of Essy's vector database. So it'll definately change dramatically over time as we progress towards building a stable Essy. We'll use the popular Chroma and build a RAG expert to begin. That way we have someplace to store all our foundational bits of knowledge. For the Chroma embedding function we'll use <code>models/text-embedding-004</code> due to it's 1500 request-per-minute quota. We'll need to be mindful of the smaller 2,048 token input. Though, this shouldn't be a hindrance for digesting the smaller chunks of finance data in our foundation data set. For the augmented generation phase we'll use <code>models/gemini-2.0-flash</code> variants due to it's 1500 request-per-day quota.
</span>

In [11]:
# An embedding function based on text-embedding-004.
class GeminiEmbeddingFunction:
    document_mode = True  # Generate embeddings for documents (T,F), or queries (F,F).
    semantic_mode = False # Semantic text similarity mode is exclusive (F,T).
    
    def __init__(self, genai_client, semantic_mode: bool = False):
        self.client = genai_client
        if semantic_mode:
            self.document_mode = False
            self.semantic_mode = True

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def __embed__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        elif not self.document_mode and not self.semantic_mode:
            embedding_task = "retrieval_query"
        elif not self.document_mode and self.semantic_mode:
            embedding_task = "semantic_similarity"
        partial = self.client.models.embed_content(
            model=embed_model,
            contents=input,
            config=types.EmbedContentConfig(task_type=embedding_task))
        return [e.values for e in partial.embeddings]
    
    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def __call__(self, input: Documents) -> Embeddings:
        try:
            response = []
            for i in range(0, len(input), 100):  # Gemini max-batch-size is 100.
                response += self.__embed__(input[i:i + 100])
            return response
        except Exception as e:
            print(f"caught exception of type {type(e)}\n{e}")
            return None

    def sts_between(self, content: list) -> float:
        df = pandas.DataFrame(self(content), index=content)
        score = df @ df.T
        return score.iloc[0].iloc[1]

In [12]:
# An implementation of Retrieval-Augmented Generation.
# - using Chroma and text-embedding-004 for storage and retrieval
# - using gemini-2.0-flash for augmented generation
class RetrievalAugmentedGenerator:
    chroma_client = chromadb.PersistentClient(path="vector_db")
    config_temp = types.GenerateContentConfig(temperature=0.0)

    def __init__(self, genai_client, collection_name):
        self.client = genai_client
        self.embed_fn = GeminiEmbeddingFunction(genai_client)
        self.db = self.chroma_client.get_or_create_collection(
            name=collection_name, 
            embedding_function=self.embed_fn, 
            metadata={"hnsw:space": "cosine"})

    def add_documents_list(self, docs: list):
        self.embed_fn.document_mode = True # Switch to document mode.
        ids = list(map(str, range(self.db.count(), self.db.count()+len(docs))))
        metas=[{"source": doc.metadata["source"]} for doc in docs]
        content=[doc.page_content for doc in docs]
        tqdm(self.db.add(ids=ids, documents=content, metadatas=metas), desc="Generate document embedding")

    def add_api_document(self, query: str, api_response: str, topic: str, source: str = "add_api_document"):
        self.embed_fn.document_mode = True # Switch to document mode.
        splitter = RecursiveJsonSplitter(max_chunk_size=2000) # chunk by token limit of models/text-embedding-004
        docs = splitter.create_documents(texts=[api_response], convert_lists=True)
        ids = list(map(str, range(self.db.count(), self.db.count()+len(docs))))
        content = [json.dumps({"question": query, "answer": doc.page_content}) for doc in docs]
        metas = [{"source": source, "topic": topic}]*len(docs)
        tqdm(self.db.add(ids=ids, documents=content, metadatas=metas), desc="Generate api embedding")

    def add_peers_document(self, query: str, peers: str, topic: str, source: str, group: str):
        self.embed_fn.document_mode = True # Switch to document mode.
        document = [{"question": query, "answer": peers}]
        tqdm(self.db.add(ids=str(self.db.count()), 
                             documents=json.dumps(document), 
                             metadatas=[{"source": source,  "topic": topic, "group": group}]), 
             desc="Generate peers embedding")

    def get_peers_document(self, query: str, topic: str, group: str):
        return self.get_documents_list(query, where={"$and": [{"group" : group}, {"topic": topic}]})

    def add_quote_document(self, query: str, quote: str, topic: str, timestamp: int, source: str):
        self.embed_fn.document_mode = True # Switch to document mode.
        document = [{"question": query, "answer": quote}]
        tqdm(self.db.add(ids=str(self.db.count()), 
                             documents=json.dumps(document), 
                             metadatas=[{"source": source,  "topic": topic, "timestamp": timestamp}]), 
             desc="Generate quote embedding")

    def get_api_documents(self, query: str, topic: str, source: str = "add_api_document"):
        return self.get_documents_list(query, where={"$and": [{"source" : source}, {"topic": topic}]})

    def query_api_documents(self, query: str, topic: str, source: str = "add_api_document"):
        return self.generate_answer(query, where={"$and": [{"source" : source}, {"topic": topic}]})

    def add_grounded_document(self, query: str, topic: str, result):
        self.embed_fn.document_mode = True # Switch to document mode.
        chunks = result.candidates[0].grounding_metadata.grounding_chunks
        supports = result.candidates[0].grounding_metadata.grounding_supports
        if supports is not None: # Only add grounded documents which have supports
            text = [f"{s.segment.text}" for s in supports]
            source = [f"{c.web.title}" for c in chunks]
            score = [f"{s.confidence_scores}" for s in supports]
            document = [{"text": ", ".join(text)}]
            tqdm(self.db.add(ids=str(self.db.count()), 
                             documents=json.dumps(document), 
                             metadatas=[{"source": ", ".join(source), 
                                         "confidence_score": ", ".join(score), 
                                         "topic": topic,
                                         "question": query}]), 
                 desc="Generate grounding embedding")

    def get_grounding_documents(self, query: str, topic: str):
        self.embed_fn.document_mode = False # Switch to query mode.
        return self.db.get(where={"$and": [{"question" : query}, {"topic": topic}]})
            
    def add_wiki_documents(self, title: str, documents: list):
        self.embed_fn.document_mode = True # Switch to document mode.
        result = self.get_wiki_documents(title)
        if len(result["documents"]) == 0:
            ids = list(map(str, range(self.db.count(), self.db.count()+len(documents))))
            metas=[{"title": title, "source": "add_wiki_documents"}]*len(documents)
            tqdm(self.db.add(ids=ids, documents=documents, metadatas=metas), desc="Generate wiki embeddings")

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def generate_with_wiki_passages(self, query: str, title: str, passages: list):
        return self.generate_answer(query, where={"title": title}, passages=passages)
    
    def get_wiki_documents(self, title: Optional[str] = None):
        self.embed_fn.document_mode = False # Switch to query mode.
        if title is None:
            return self.db.get(where={"source": "add_wiki_document"})
        else:
            return self.db.get(where={"title": title})

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_documents_list(self, query: str, max_sources: int = 10, where: Optional[dict] = None):
        self.embed_fn.document_mode = False # Switch to query mode.
        result = self.db.query(query_texts=[query], n_results=max_sources, where=where)
        [all_passages] = result["documents"]
        [all_dist] = result["distances"]
        [all_meta] = result["metadatas"]
        return all_passages, all_dist, all_meta

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_exchanges_csv(self, query: str):
        return self.generate_answer(query, max_sources=100, where={"source": "exchanges.csv"})

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def generate_answer(self, query: str, max_sources: int = 10, 
                        where: Optional[dict] = None, passages: Optional[list] = None):
        passage_list, dist_list, meta_list = self.get_documents_list(query, max_sources, where)
        query_oneline = query.replace("\n", " ")
        prompt = f"""You're an expert writer. You understand how to interpret html and markdown. You will accept the
        question below and answer based only on the passages. Never mention the passages in your answers. Be sure to 
        respond in concise sentences. Include all relevant background information when possible. If a passage is not 
        relevant to the answer you must ignore it. If no passage answers the question respond with: I don't know.

        QUESTION: {query_oneline}
        
        """
        # Add the retrieved documents to the prompt.
        for passage in passage_list if passages is None else passage_list + passages:
            passage_oneline = passage.replace("\n", " ")
            prompt += f"PASSAGE: {passage_oneline}\n"
    
        return self.client.models.generate_content(model=project_model, 
                                                   config=self.config_temp, 
                                                   contents=prompt)

In [13]:
# An implementation of Wiki-Grounding Generation.
# - using gemini-2.0-flash for response generation
# - using a RAG-implementation to store groundings
# - create new groundings by similarity to topic
# - retrieve existing groundings by similarity to topic
class WikiGroundingGenerator:   
    def __init__(self, genai_client, rag_impl):
        self.client = genai_client
        self.rag = rag_impl
        with warnings.catch_warnings():
            warnings.simplefilter("ignore") # suppress beta-warning
            self.splitter = HTMLSemanticPreservingSplitter(
                headers_to_split_on=[("h2", "Main Topic"), ("h3", "Sub Topic")],
                separators=["\n\n", "\n", ". ", "! ", "? "],
                max_chunk_size=2000, # chunk by token limit of models/text-embedding-004
                chunk_overlap=50,
                preserve_links=True,
                preserve_images=True,
                preserve_videos=True,
                preserve_audio=True,
                elements_to_preserve=["table", "ul", "ol", "code"],
                denylist_tags=["script", "style", "head"],
                custom_handlers={"code": self.code_handler},
            )

    def generate_answer(self, query: str, topic: str):
        result = self.rag.get_wiki_documents(topic)
        if len(result["documents"]) > 0:
            return self.rag.generate_with_wiki_passages(query, topic, result["documents"]).text
        else:
            pages = wikipedia.search(topic + " company")
            if len(pages) > 0:
                p_topic_match = 0.80
                for i in range(len(pages)):
                    if tqdm(self.get_topic_similarity(topic, pages[i]) > p_topic_match, 
                            desc= "Score wiki search by similarity to topic"):
                        request = requests.get(f"https://en.wikipedia.org/wiki/{pages[i]}")
                        documents = [document.page_content for document in self.splitter.split_text(request.text)]
                        self.rag.add_wiki_documents(topic, documents)
                        return self.rag.generate_with_wiki_passages(query, topic, documents).text

    def code_handler(self, element: Tag) -> str:
        data_lang = element.get("data-lang")
        code_format = f"<code:{data_lang}>{element.get_text()}</code>"
        return code_format

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_topic_similarity(self, topic: str, page: str):
        return GeminiEmbeddingFunction(client, semantic_mode = True).sts_between([topic + " company", page])

In [14]:
# An implementation of Grounding Generation.
# - using gemini-2.0-flash with GoogleSearch tool for response generation
# - using a RAG-implementation to store groundings
# - create new groundings by exact match to topic
# - retrieve existing groundings by similarity to topic
class GroundingGenerator:
    config_ground = types.GenerateContentConfig(
        tools=[types.Tool(google_search=types.GoogleSearch())],
        temperature=0.0
    )
    
    def __init__(self, genai_client, rag_impl):
        self.client = genai_client
        self.rag = rag_impl

    def generate_answer(self, query: str, topic: str):
        docs = self.rag.get_grounding_documents(query, topic)
        if len(docs["documents"]) > 0:
            for i in range(len(docs["metadatas"])):
                doc = docs["documents"][i]
                meta_q = docs["metadatas"][i]["question"]
                p_ground_match = 0.95 # This can be really high ~ 95-97%
                if tqdm(self.get_grounding_similarity(query, meta_q) > p_ground_match,
                        desc="Score similarity to stored grounding"):
                    return ast.literal_eval(doc)[0]["text"]
        return self.get_grounding(query, topic)

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_grounding_similarity(self, question: str, compare: str):
        return GeminiEmbeddingFunction(client, semantic_mode = True).sts_between([question, compare])

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_grounding(self, query: str, topic: str):
        contents = [types.Content(role="user", parts=[types.Part(text=query)])]
        contents += f"""
        You're a search assistant that provides grounded answers to questions about {topic}. You will provide only 
        results that discuss {topic}. Be brief and specific in answering and omit extra details.
        If an answer is not possible respond with: I don't know."""
        response = self.client.models.generate_content(
            model=project_model, 
            config=self.config_ground, 
            contents=contents)
        if response.candidates[0].grounding_metadata.grounding_supports is not None:
            if topic.replace("'", "") not in response.text: # Exact topic match required
                return "I don't know." # Workaround a bug in gemini-2.0-flash (MGM Studio becomes MGM Resorts)
            else:
                self.rag.add_grounded_document(query, topic, response)
                return response.text
        return "I don't know." # Empty grounding_supports means grounding not possible for query.

# Testing the RAG Implementation

<span style="font-size:18px;">
Let's load some test data and see what the RAG can do. The test data is a CSV file containing stock market exchange data. It includes the market id code, name, locale, and operating hours. The import will use CSVLoader from <code>langchain-community</code> to parse the exchange data into Documents that our RAG can ingest.
</span>

In [15]:
# Load the exchange data from source csv.
# - Identifies exchanges by a 1-2 letter code which can be used to filter response data.
# - Also maps the exchange code to exchange details.
df = pandas.read_csv("/kaggle/input/exchanges/exchanges_src.csv").drop(["close_date"], axis=1).fillna("")
df.to_csv("exchanges.csv", index=False)
exchanges = CSVLoader(file_path="exchanges.csv", encoding="utf-8", csv_args={"delimiter": ","}).load()

# Prepare a RAG tool for use and add the exchange data.
tool_rag = RetrievalAugmentedGenerator(client, "finance")
tool_rag.add_documents_list(exchanges)

# Prepare a the grounding tools for use.
tool_wiki = WikiGroundingGenerator(client, tool_rag)
tool_ground = GroundingGenerator(client, tool_rag)

Generate document embedding: 0it [00:00, ?it/s]


<span style="font-size:18px;">
Now that the data is loaded lets ask our RAG to perform some augmenting. We can ask it to perform all sorts of useful tasks. We'll generate some useful reusable data structures and check to make sure it can answer important questions. The exchanges all have id's which are used to filter the realtime data. So we'll make sure the RAG know how to create this mapping. We'll also check it's awareness of operating hours. After all, Essy, doesn't mindlessly hammer away at api's when no new data is available.
</span>

In [16]:
# The RAG tool is a helpful expert.

response = tool_rag.get_exchanges_csv("""Give me a dictionary in string form. It must contain key:value pairs mapping 
                                         exchange code to name. Just the dictionary string in pretty form.""")
print(response.text)

response = tool_rag.get_exchanges_csv("""What is the Germany exchange code? Return only the exchange codes as a simple
                                         comma separated value that I can copy.""")
print(response.text)

response = tool_rag.get_exchanges_csv("What are the Germany exchanges and thier corresponding exchange codes?")
print(response.text, "\n")

response = tool_rag.generate_answer("What are Google's stock ticker symbols?")
print(response.text, "\n")

response = tool_rag.get_exchanges_csv("What are the US exchange operating hours?")
print(response.text, "\n")

est = pytz.timezone('US/Eastern') # Exchanges data is in eastern time.
response = tool_rag.get_exchanges_csv(
    f"""Answer based on your knowledge of exchange operating hours.
        Do not answer in full sentences. Omit all chat and provide the answer only.
        All exchanges are open on weekdays. Weekdays are: Mon, Tue, Wed, Thu, Fri.
        Exchanges open and close on weekdays.
        
        The current date and time is: {datetime.now(est).strftime('%c')}
        
        When was the US exchange's last operating hours? Exclude weekends.
        Provide just the close. Include post-market hours.
        Answer with a date that uses this format: '%a %b %d %X %Y'.""")
print(response.text)

```
{
    "SC": "BOERSE_FRANKFURT_ZERTIFIKATE",
    "SX": "DEUTSCHE BOERSE Stoxx",
    "HK": "HONG KONG EXCHANGES AND CLEARING LTD",
    "DB": "DUBAI FINANCIAL MARKET",
    "NZ": "NEW ZEALAND EXCHANGE LTD",
    "QA": "QATAR EXCHANGE",
    "KS": "KOREA EXCHANGE (STOCK MARKET)",
    "SW": "SWISS EXCHANGE",
    "DU": "BOERSE DUESSELDORF",
    "BC": "BOLSA DE VALORES DE COLOMBIA",
    "KQ": "KOREA EXCHANGE (KOSDAQ)",
    "SN": "SANTIAGO STOCK EXCHANGE",
    "SI": "SINGAPORE EXCHANGE",
    "AD": "ABU DHABI SECURITIES EXCHANGE",
    "CO": "OMX NORDIC EXCHANGE COPENHAGEN A/S",
    "L": "LONDON STOCK EXCHANGE",
    "ME": "MOSCOW EXCHANGE",
    "TO": "TORONTO STOCK EXCHANGE",
    "BD": "BUDAPEST STOCK EXCHANGE",
    "TG": "DEUTSCHE BOERSE TradeGate",
    "US": "US exchanges (NYSE, Nasdaq)",
    "TW": "TAIWAN STOCK EXCHANGE",
    "JK": "INDONESIA STOCK EXCHANGE",
    "SZ": "SHENZHEN STOCK EXCHANGE",
    "VS": "NASDAQ OMX VILNIUS",
    "MX": "BOLSA MEXICANA DE VALORES (MEXICAN STOCK EXCHANGE)",
 

<span style="font-size:18px;">
Excellent! Though, despite my best effort I could not convince Gemini to apply date correction (during chaining) based on holiday. It simply wasn't stable enough to be useful. I would either have to add a holiday data set, or (what I chose) apply a quick temporary fix. A real-time API endpoint may fail due to a holiday being selected as the date. If that happens I'll just retry Thursday if the failure happened on Friday, likewise choosing Friday if the failure happened on Monday. Crude but simple for foundational purposes.
</span>

# Declaring the Function Calling Metadata

<span style="font-size:18px;">
Our Function Calling expert will chain together the other experts we've implemented thus far. It also provides the final response through augmentation. This time using the tools as a source of grounding truth. It'd like to say it's all truth organised by topic and other metadata. It's still a precarious situation if Essy incidently chains into mining data on another topic. We want Amazon to be the owner of MGM Studio's not MGM Resorts International. We also don't want a summary to include another company unless that company is a peer.
</span>

<span style="font-size:18px;">
The function calling metadata is thus extremely important. It needs to combine our other experts with the real-time api's data. Essy will use two API providers as sources of finance data. The primary motivation being that each provider has limits in their own way, yet both are useful in their own own way. This is useful anywhere you need a broad spectrum of sources of truth. At metadata creation I'll adopt the naming convention of appending the provider (if any) id. This helps keep functions more understandable when you know which provider you're dealing with.
</span>

In [17]:
# Declare callable functions using OpenAPI schema.
get_symbol_1 = types.FunctionDeclaration(
    name="get_symbol_1",
    description="""Search for the stock ticker symbol of a given company, security, isin or cusip. Each ticker
                   entry provides a description, symbol, and asset type. If this doesn't help you should try 
                   calling get_wiki_tool_response next.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": """The company, security, isin or cusip to search for a symbol."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["q", "exchange", "query"]
    }
)

get_name_1 = types.FunctionDeclaration(
    name="get_name_1",
    description="""Search for the name associated with a stock ticker or symbol's company, security, isin or cusip. 
    Each ticker entry provides a description, matching symbol, and asset type.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": """The symbol or ticker to search for."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            },
            "company": {
                "type": "string",
                "description": "The company you're searching for."
            }
        },
        "required": ["q", "exchange", "query", "company"]
    }
)

get_symbol_quote_1 = types.FunctionDeclaration(
    name="get_symbol_quote_1",
    description="""Search for the current price or quote of a stock ticker or symbol. The response is
                   provided in json format. Each response contains the following key-value pairs:
                   
                   c: Current price,
                   d: Change,
                  dp: Percent change,
                   h: High price of the day,
                   l: Low price of the day,
                   o: Open price of the day,
                  pc: Previous close price,
                   t: Epoch timestamp of price in seconds.

                   Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "The stock ticker symbol for a company, security, isin, or cusip." 
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            },
            "exchange": {
                "type": "string",
                "description": "The exchange code used to filter quotes. This must always be 'US'."
            }
        },
        "required": ["symbol", "query", "exchange"]
    }
)

get_local_datetime_1 = types.FunctionDeclaration(
    name="get_local_datetime_1",
    description="""Converts an array of timestamps from epoch time to the local timezone format. The result is an array
                   of date and time in locale appropriate format. Suitable for use in a locale appropriate response.
                   Treat this function as a vector function. Always prefer to batch timestamps for conversion. Use this
                   function to format date and time in your responses.""",
    parameters={
        "type": "object",
        "properties": {
            "t": {
                "type": "array",
                "description": """An array of timestamps in seconds since epoch to be converted. The order of
                                  timestamps matches the order of conversion.""",
                "items": {
                    "type": "integer"
                }
            }
        },
        "required": ["t"]
    }
)

get_market_status_1 = types.FunctionDeclaration(
    name="get_market_status_1",
    description="""Get the current market status of global exchanges. Includes whether exchanges are open or closed.  
                   Also includes holiday details if applicable. The response is provided in json format. Each response 
                   contains the following key-value pairs:

                   exchange: Exchange code,
                   timezone: Timezone of the exchange,
                    holiday: Holiday event name, or null if it's not a holiday,
                     isOpen: Whether the market is open at the moment,
                          t: Epoch timestamp of status in seconds (Eastern Time),
                    session: The market session can be 1 of the following values: 
                    
                    pre-market,regular,post-market when open, or null if closed.
                    
                    Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. The default if omitted is 'US' for the 
                                  US exchanges. A dictionary mapping supported exchange codes (key) to their 
                                  description (value) can be obtained from get_exchange_codes_1. Search the values for
                                  a matching exchange code if unsure."""
            }
        },
        "required": ["exchange"]
    }
)

get_company_peers_1 = types.FunctionDeclaration(
    name="get_company_peers_1",
    description="""Search for a company's peers. Returns a list of peers operating in the same country and in the same
                   sector, industry, or subIndustry. Each response contains the following key-value pairs: 
                   
                   symbol: The company's stock ticker symbol, 
                   peers: A list containing the peers.
                   
                   Each peers entry contains the following key-value pairs:
                   
                   symbol: The peer company's stock ticker symbol, 
                   name: The peer company's name.
                   
                   Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "The stock ticker symbol of a company to obtain peers."
            },
            "grouping": {
                "type": "string",
                "description": """Specify the grouping category for choosing peers. When not specified the default
                                  category is subIndustry. This parameter may be one of the following values: 
                                  sector, industry, subIndustry."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "grouping", "exchange", "query"]
    }
)

get_exchange_codes_1 = types.FunctionDeclaration(
    name="get_exchange_codes_1",
    description="""Get a dictionary mapping all supported exchange codes to their names."""
)

get_exchange_code_1 = types.FunctionDeclaration(
    name="get_exchange_code_1",
    description="""Search for the exchange code to use when filtering by exchange. The result will be one or
                   more exchange codes provided as a comma-separated string value.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": "Specifies which exchange code to search for."
            }
        },
        "required": ["q"]
    }
)

get_financials_1 = types.FunctionDeclaration(
    name="get_financials_1",
    description="""Get company basic financials such as margin, P/E ratio, 52-week high/low, etc. Parse the response for 
                   key-value pairs in json format and interpret their meaning as stock market financial indicators.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "metric": {
                "type": "string",
                "description": "It must always be declared as the value 'all'"
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "metric", "query"]
    }
)

get_daily_candlestick_2 = types.FunctionDeclaration(
    name="get_daily_candlestick_2",
    description="""Get a historical daily stock ticker candlestick / aggregate bar (OHLC). 
                   Includes historical daily open, high, low, and close prices. Also includes historical daily trade
                   volume and pre-market/after-hours trade prices. It does not provide today's data until after 
                   11:59PM Eastern Time.""",
    parameters={
        "type": "object",
        "properties": {
            "stocksTicker": {
                "type": "string",
                "description": "The stock ticker symbol of a company to search for.",
            },
            "date": {
                "type": "string",
                "format": "date-time",
                "description": """The date of the requested candlestick in format YYYY-MM-DD. The default is one 
                                  weekday prior to get_last_market_close (excluding weekends). This date must never 
                                  be more recent than the default. Replace more recent dates with the default."""
            },
            "adjusted": {
                "type": "string",
                "description": """May be true or false. Indicated whether or not the results are adjusted for splits. 
                                  By default, results are adjusted. Set this to false to get results that are NOT 
                                  adjusted for splits."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["stocksTicker", "date", "adjusted", "query"]
    },
)

get_company_news_1 = types.FunctionDeclaration(
    name="get_company_news_1",
    description="Retrieve the most recent news articles related to a specified ticker.",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company.",
            },
            "from": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD. It must be older than the parameter 'to'. The default
                                  value is one month ago."""
            },
            "to": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD. It must be more recent than the parameter 'from'. The
                                  default value is today's date."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "from", "to", "query"]
    },
)

get_custom_candlestick_2 = types.FunctionDeclaration(
    name="get_custom_candlestick_2",
    description="""Get a historical stock ticker candlestick / aggregate bar (OHLC) over a custom date range and 
                   time interval in Eastern Time. Includes historical open, high, low, and close prices. Also 
                   includes historical daily trade volume and pre-market/after-hours trade prices. It does not
                   include today's open, high, low, or close until after 11:59PM Eastern Time.""",
    parameters={
        "type": "object",
        "properties": {
            "stocksTicker": {
                "type": "string",
                "description": "The stock ticker symbol of a company to search for.",
            },
            "multiplier": {
                "type": "integer",
                "description": "This must be 1 unless told otherwise."
            },
            "timespan": {
                "type": "string",
                "description": """The size of the candlestick's time window. This is allowed to be one of the following:
                                  second, minute, hour, day, week, month, quarter, or year. The default value is day."""
            },
            "from": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be older than the parameter 'to'. The default
                                  value is one-month ago from today's date."""
            },
            "to": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be more recent than the parameter 'from'. The 
                                  default is one weekday prior to get_last_market_close (excluding weekends).
                                  Replace more recent dates with the default."""
            },
            "adjusted": {
                "type": "string",
                "description": """May be true or false. Indicated whether or not the results are adjusted for splits. 
                                  By default, results are adjusted. Set this to false to get results that are NOT 
                                  adjusted for splits."""
            },
            "sort": {
                "type": "string",
                "description": """May be one of asc or desc. asc will sort by timestmap in ascending order. desc will
                                  sort by timestamp in descending order."""
            },
            "limit": {
                "type": "integer",
                "description": """Set the number of base aggregates used to create this candlestick. This must be 5000 
                                  unless told to limit base aggregates to something else."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["stocksTicker", "multiplier", "timespan", "from", "to", "query", "adjusted", "sort", "limit"]
    },
)

get_last_market_close = types.FunctionDeclaration(
    name="get_last_market_close",
    description="""Get the date and time of the US exchange market's last close. Provides the last US market close in 
                   locale appropriate format."""
)

get_ticker_overview_2 = types.FunctionDeclaration(
    name="get_ticker_overview_2",
    description="""Retrieve comprehensive details for a single ticker symbol. It's a deep look into a company’s 
    fundamental attributes, including its primary exchange, standardized identifiers (CIK, composite FIGI, 
    share class FIGI), market capitalization, industry classification, and key dates. Also includes branding assets in
    the form of icons and logos.
    """,
    parameters={
        "type": "object",
        "properties": {
            "ticker": {
                "type": "string",
                "description": "Stock ticker symbol of a company."
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["ticker", "query"]
    }
)

get_recommendation_trends_1 = types.FunctionDeclaration(
    name="get_recommendation_trends_1",
    description="""Get the latest analyst recommendation trends for a company.
                The data includes the latest recommendations as well as historical
                recommendation data for each month. The data is classified according
                to these categories: strongBuy, buy, hold, sell, and strongSell.
                The date of a recommendation indicated by the value of 'period'.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "query"]
    }
)

get_news_with_sentiment_2 = types.FunctionDeclaration(
    name="get_news_with_sentiment_2",
    description="""Retrieve the most recent news articles related to a specified ticker. Each article includes 
                   comprehensive coverage. Including a summary, publisher information, article metadata, 
                   and sentiment analysis.""",
    parameters={
        "type": "object",
        "properties": {
            "ticker": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "published_utc": {
                "type": "string",
                "format": "date-time",
                "description": """Omit this parameter unless you're told told to filter by published_utc."""
            },
            "order": {
                "type": "string",
                "description": """Must be desc for descending order, or asc for ascending order.
                                  When order is not specified the default is descending order.
                                  Ordering will be based on the parameter: sort."""
            },
            "limit": {
                "type": "integer",
                "description": """This must be 100 unless told to limit news results to something else."""
            },
            "sort": {
                "type": "string",
                "description": """The sort field used for ordering. This value must
                                  always be published_utc."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["ticker", "order", "limit", "sort", "query"]
    }
)

get_rag_tool_response = types.FunctionDeclaration(
    name="get_rag_tool_response",
    description="""A database containing useful financial information. Always check here for answers first.""",
    parameters={
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "A question needing an answer. Asked as a simple string."
            }
        }
    }
)

get_wiki_tool_response = types.FunctionDeclaration(
    name="get_wiki_tool_response",
    description="""Answers questions that still have unknown answers. Retrieve a wiki page related to a company, 
                   product, or service. Each web page includes detailed company information, financial indicators, 
                   tickers, symbols, history, and products and services.""",
    parameters={
        "type": "object",
        "properties": {
            "id": {
                "type": "string",
                "description": "The question's company or product. Just the name and no other details."
            },
            "q": {
                "type": "string",
                "description": "The complete, unaltered, query string."
            }
        },
        "required": ["id", "q"]
    }
)

get_search_tool_response = types.FunctionDeclaration(
    name="get_search_tool_response",
    description="Answers questions that still have unknown answers. Use it after checking all your other tools.",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": "The question needing an answer. Asked as a simple string."
            },
            "id": {
                "type": "string",
                "description": "The question's company or product. In one word. Just the name and no other details."
            }
        },
        "required": ["q", "id"]
    }
)

# Implementing the Function Calls

<span style="font-size:18px;">
One downside of this part being the main part was the lack of time to refactor this part more. Our formative Essy implements as much useful data from two finacial APIs. In order to use it you will need to declare secrets for <a class="anchor-link" href="https://finnhub.io/dashboard">Finnhub</a> and <a class="anchor-link" href="https://polygon.io/dashboard">Polygon</a> finance APIs. Register at their respective sites for your free API key. Then import the secret using the same method as how you setup Google's API key.
</span>

In [18]:
# Implement the callable functions and the function handler.

def ask_rag_tool(content):
    return tool_rag.generate_answer(content["question"]).text

def ask_wiki_tool(content):
    return tool_wiki.generate_answer(content["q"], content["id"])

def ask_search_tool(content):
    return tool_ground.generate_answer(content["q"], content["id"])

def rag_exchange_codes_1(content):
    response = tool_rag.get_exchanges_csv("""Give me a dictionary in string form. It must contaihttps://api.polygon.io/v3/reference/tickers/AAPL?apiKey=4xJe226Z23RZmEc1bN8az1zz4pmNWdOpn key:value pairs 
                                             mapping exchange code to name. Just the dictionary string.
                                             Omit all other information or details. Do not chat or use sentences.""")
    codes = list(ast.literal_eval(response.text.strip("\`")).items())
    return codes

def rag_exchange_code_1(content):
    codes = tool_rag.get_exchanges_csv(f"""What is the {content} exchange code? Return only the exchange codes 
                                           as a list in string form. Just the list string.
                                           Omit all other information or details. Do not chat or use sentences.""").text
    return ast.literal_eval(codes)

def rag_last_market_close(content):
    est = pytz.timezone('US/Eastern') # Exchanges data is in eastern time.
    return tool_rag.get_exchanges_csv(
        f"""Answer based on your knowledge of exchange operating hours.
        Do not answer in full sentences. Omit all chat and provide the answer only.
        All exchanges are open on weekdays. Weekdays are: Mon, Tue, Wed, Thu, Fri.
        Exchanges open and close on weekdays.
        
        The current date and time is: {datetime.now(est).strftime('%c')}
        
        When was the US exchange's last operating hours? Exclude weekends.
        Provide just the close. Include post-market hours.
        Answer with a date that uses this format: '%a %b %d %X %Y'.""").text

@retry.Retry(
    predicate=is_retriable,
    initial=2.0,
    maximum=64.0,
    multiplier=2.0,
    timeout=600,
)
def get_similarity_score(content):
    return GeminiEmbeddingFunction(client, semantic_mode = True).sts_between(content)
    
def impl_get_symbol_1(content, by_name: bool = True):
    response = tool_rag.get_api_documents(content["query"], content["q"], "get_symbol_1")
    if len(response[0]) == 0: # index [0] for document content
        url = f"https://finnhub.io/api/v1/search?q={content['q']}&exchange={content['exchange']}&token={FINNHUB_API_KEY}"
        try:
            response = json.loads(requests.get(url).text)
        except:
            return "I don't know."
        else:
            matches = []
            max_failed_match = len(response["result"]) if not by_name else 3
            p_desc_match = 0.80
            p_symb_match = 0.95
            if response["count"] > 0:
                for result in tqdm(response["result"], desc="Score similarity to query"):
                    if max_failed_match > 0:
                        desc = [content['q'].upper(), result["description"].split("-", -1)[0]]
                        symb = [content['q'].upper(), result["symbol"]]
                        if by_name and get_similarity_score(desc) > p_desc_match: 
                            matches.append(result["symbol"])
                        elif not by_name and get_similarity_score(symb) > p_symb_match:
                            matches.append(result["description"])
                            max_failed_match = 0
                        else:
                            max_failed_match -= 1
            if len(matches) > 0:
                tool_rag.add_api_document(content["query"], matches, content["q"], "get_symbol_1")
                return matches
            else:
                return "I don't know."
    else:
        doc = ast.literal_eval(response[0][0])[0]
        return doc["answer"]

def impl_get_name_1(content):
    return impl_get_symbol_1(content, by_name = False)

def impl_get_quote_1(content):
    quotes = tool_rag.get_api_documents(content["query"], content["symbol"], "get_quote_1")
    isOpen = dict(impl_get_market_status_1(content))["isOpen"]
    if len(quotes[0]) == 0 or isOpen: 
        return get_current_price_1(content)
    else:
        last_close = parse(rag_last_market_close(content)).timestamp()
        for quote in quotes[2]: # index [2] for metadata
            if quote["timestamp"] >= last_close:
                return quotes
        return get_current_price_1(content)

def get_current_price_1(content):
    url = f"https://finnhub.io/api/v1/quote?symbol={content['symbol']}&token={FINNHUB_API_KEY}"
    # This is a high-demand endpoint. Expect random failure under heavy (free) use.
    try:
        response = json.loads(requests.get(url).text)
    except:
        return "I don't know."
    else:
        if len(response) > 0 and response["t"] > 0:
            tool_rag.add_quote_document(content["query"], response, content["symbol"], response["t"], "get_quote_1")
            return list(response.items())
        return "I don't know."

def impl_get_market_status_1(content):
    url = f"https://finnhub.io/api/v1/stock/market-status?exchange={content['exchange']}&token={FINNHUB_API_KEY}"
    try:
        response = json.loads(requests.get(url).text)
    except:
        return "I don't know."
    else:
        if len(response) > 0:
            return list(response.items())
        return "I don't know."

def impl_get_peers_1(content):
    docs = tool_rag.get_peers_document(content["query"], content["symbol"], content['grouping'])
    if len(docs[0]) == 0: # index [0] for document content
        url = f"https://finnhub.io/api/v1/stock/peers?symbol={content['symbol']}&grouping={content['grouping']}&token={FINNHUB_API_KEY}"
        try:
            peers = json.loads(requests.get(url).text)
        except:
            return "I don't know."
        else:
            if len(peers) > 0:
                names = []
                for peer in peers:
                    if peer == content["symbol"]:
                        continue # skip including the query symbol in peers (included in metadata anyway)
                    name_lookup = dict(q=peer, exchange=content["exchange"], query=content["query"])
                    name = impl_get_name_1(name_lookup)
                    if name != "I don't know.":
                        p = {"symbol": peer, "name": name}
                        names.append(p)
                peers = {"symbol": content["symbol"], "peers": names}
                tool_rag.add_peers_document(content["query"], peers, content["symbol"], "get_peers_1", content['grouping'])
                return list(peers.items())
            return "I don't know."
    else:
        peers = ast.literal_eval(docs[0][0])[0]["answer"] # The first document should be most relevant.
        return list(peers.items())

def impl_local_datetime_1(content):
    local_t = []
    for timestamp in content["t"]:
        local_t.append(local_date_from_epoch(timestamp))
    return local_t

def local_date_from_epoch(timestamp):
    est = pytz.timezone('US/Eastern') # Exchanges data is in eastern time.
    if len(str(timestamp)) == 13:
        return datetime.fromtimestamp(timestamp/1000, tz=est).strftime('%c')
    else:
        return datetime.fromtimestamp(timestamp, tz=est).strftime('%c')

def impl_get_financials_1(content):
    fins = tool_rag.get_api_documents(content["query"], content["symbol"], "get_financials_1")
    if len(fins[0]) == 0:
        url = f"https://finnhub.io/api/v1/stock/metric?symbol={content['symbol']}&metric={content['metric']}&token={FINNHUB_API_KEY}"
        try:
            fin = json.loads(requests.get(url).text)
        except:
            return "I don't know."
        else:
            if not fin:
                return "I don't know."
            tool_rag.add_api_document(content["query"], fin, content["symbol"], "get_financials_1")
            return list(fin.items())
    return fins

def impl_get_news_1(content):
    #news = tool_rag.get_api_documents(content["query"], content["symbol"], "get_news_1")
    #if len(news[0]) == 0:
        url = f"https://finnhub.io/api/v1/company-news?symbol={content['symbol']}&from={content['from']}&to={content['to']}&token={FINNHUB_API_KEY}"
        try:
            news = json.loads(requests.get(url).text)
        except:
            return "I don't know."
        else:
            if len(news) == 0:
                return "I don't know."
            #tool_rag.add_api_document(content["query"], news, content["symbol"], "get_news_1")
            return list(news.items())
    #return news

def impl_daily_candle_2(content):
    daily_candle = tool_rag.get_api_documents(content["query"], content["stocksTicker"], "daily_candle_2")
    if len(daily_candle[0]) == 0:
        url = f"https://api.polygon.io/v1/open-close/{content['stocksTicker']}/{content['date']}?adjusted={content['adjusted']}&apiKey={POLYGON_API_KEY}"
        try:
            request = requests.get(url)
            daily_candle = ast.literal_eval(request.text)
        except:
            return f"I don't know. Endpoint returned status {request.status_code}"
        else:
            if daily_candle["status"] in ["OK","DELAYED"]:
                tool_rag.add_api_document(content["query"], daily_candle, content["stocksTicker"], "daily_candle_2")
                return list(daily_candle.items())
            else:
                date = parse(content["date"])
                new_date = None
                if date.weekday() == 4: # index 4 for friday
                    new_date = date - timedelta(days=1)
                elif date.weekday() == 0: # index 0 for monday
                    new_date = date - timedelta(days=3)
                if new_date is None:
                    return "I don't know."
                content["date"] = new_date.strftime("%Y-%m-%d")
                return impl_daily_candle_2(content)
    return daily_candle

def impl_custom_candle_2(content):
    url = f"""https://api.polygon.io/v2/aggs/ticker/{content['stocksTicker']}/range/{content['multiplier']}/{content['timespan']}/{content['from']}/{content['to']}?adjusted={content['adjusted']}&sort={content['sort']}&limit={content['limit']}&apiKey={POLYGON_API_KEY}"""
    try:
        request = requests.get(url)
        custom_candle = json.loads(request.text)
    except:
        return f"I don't know. Endpoint returned status {request.status_code}"
    else:
        if custom_candle["status"] in ["OK","DELAYED"]:
            tool_rag.add_api_document(content["query"], custom_candle, content["stocksTicker"], "custom_candle_2")
            return list(custom_candle.items())
        return "I don't know."

def impl_ticker_overview_2(content):
    overview = tool_rag.get_api_documents(content["query"], content["ticker"], "ticker_overview_2")
    if len(overview[0]) == 0:
        url = f"https://api.polygon.io/v3/reference/tickers/{content['ticker']}?apiKey={POLYGON_API_KEY}"
        try:
            request = requests.get(url)
            overview = json.loads(request.text)
        except:
            return f"I don't know. Endpoint returned status {request.status_code}"
        else:
            if overview["status"] in ["OK","DELAYED"]:
                tool_rag.add_api_document(content["query"], overview, content["ticker"], "ticker_overview_2")
                return list(overview.items())
            return "I don't know."
    return overview

def impl_trends_1(content):
    trends = tool_rag.get_api_documents(content["query"], content["symbol"], "trends_1")
    if len(trends[0]) == 0:
        url = f"https://finnhub.io/api/v1/stock/recommendation?symbol={content['symbol']}&token={FINNHUB_API_KEY}"
        try:
            trends = json.loads(requests.get(url).text)
        except:
            return "I don't know."
        else:
            if len(trends) > 0:
                tool_rag.add_api_document(content["query"], trends, content["symbol"], "trends_1")
                return trends
            return "I don't know."
    return trends

def impl_get_news_2(content):
    #news = tool_rag.get_api_documents(content["query"], content["ticker"], "get_news_2")
    #if len(news[0]) == 0:
        url = f"https://api.polygon.io/v2/reference/news?ticker={content['ticker']}&order={content['order']}&limit={content['limit']}&sort={content['sort']}&apiKey={POLYGON_API_KEY}"
        try:
            request = requests.get(url)
            news = json.loads(request.text)
        except:
            return f"I don't know. Endpoint returned status {request.status_code}"
        else:
            if news["status"] in ["OK","DELAYED"]:
                #tool_rag.add_api_document(content["query"], news, content["ticker"], "get_news_2")
                return list(news.items())
            return "I don't know."
    #return news
        
finance_tool = types.Tool(
    function_declarations=[
        get_symbol_1,
        get_name_1,
        get_symbol_quote_1,
        get_market_status_1,
        get_company_peers_1,
        get_local_datetime_1,
        get_last_market_close,
        get_exchange_codes_1,
        get_exchange_code_1,
        get_financials_1,
        get_daily_candlestick_2,
        get_custom_candlestick_2,
        get_ticker_overview_2,
        get_recommendation_trends_1,
        get_news_with_sentiment_2,
        get_rag_tool_response,
        get_wiki_tool_response,
        get_search_tool_response
    ]
)

function_handler = {
    "get_symbol_1": impl_get_symbol_1,
    "get_name_1": impl_get_name_1,
    "get_symbol_quote_1": impl_get_quote_1,
    "get_market_status_1": impl_get_market_status_1,
    "get_company_peers_1": impl_get_peers_1,
    "get_local_datetime_1": impl_local_datetime_1,
    "get_last_market_close": rag_last_market_close,
    "get_exchange_codes_1": rag_exchange_codes_1,
    "get_exchange_code_1": rag_exchange_code_1,
    "get_financials_1": impl_get_financials_1,
    "get_daily_candlestick_2": impl_daily_candle_2,
    "get_custom_candlestick_2": impl_custom_candle_2,
    "get_ticker_overview_2": impl_ticker_overview_2,
    "get_recommendation_trends_1": impl_trends_1,
    "get_news_with_sentiment_2": impl_get_news_2,
    "get_rag_tool_response": ask_rag_tool,
    "get_wiki_tool_response": ask_wiki_tool,
    "get_search_tool_response": ask_search_tool
}

In [19]:
# Define the system prompt.

instruction = f"""You are a helpful and informative bot that answers finance and stock market questions. 
Only answer the question asked and do not change topic. While the answer is still
unknown you must follow these rules for predicting function call order:

RULE#1: Always consult your other functions before get_search_tool_response.
RULE#2: Always consult get_wiki_tool_response before get_search_tool_response.
RULE#3: Always consult get_search_tool_response last.
RULE#4: Always convert timestamps with get_local_datetime_1 and use the converted date/time in your response.
RULE#5: Always incorporate as much useful information from tools and functions in your response."""

In [20]:
# Import the finance api secret keys.

POLYGON_API_KEY = UserSecretsClient().get_secret("POLYGON_API_KEY")
FINNHUB_API_KEY = UserSecretsClient().get_secret("FINNHUB_API_KEY")

In [21]:
# Implement the function calling expert.

@retry.Retry(
    predicate=is_retriable,
    initial=2.0,
    maximum=64.0,
    multiplier=2.0,
    timeout=600,
)
def send_message(prompt):
    #display(Markdown("#### Prompt"))
    #print(prompt, "\n")
    # Define the user prompt part.
    contents = [types.Content(role="user", parts=[types.Part(text=prompt)])]
    # Gemini's innate notion of current date and time is unstable.
    est = pytz.timezone('US/Eastern') # The finance api data is in eastern time.
    contents += f"""
    The current date and time is: {datetime.now(est).strftime('%c')}
    
    Give a concise, and detailed summary. Use information that you learn from the API responses.
    Use your tools and function calls according to the rules. Convert any all-upper case identifiers
    to proper case in your response. Convert any abbreviated or shortened identifiers to their full forms.
    Convert timestamps according to the rules before including them.
    """
    # Enable system prompt, function calling and minimum-randomness.
    config_fncall = types.GenerateContentConfig(
        system_instruction=instruction,
        tools=[finance_tool],
        temperature=0.0
    )
    # Handle cases with multiple chained function calls.
    function_calling_in_process = True
    while function_calling_in_process:
        # Send the user prompt and function declarations.
        response = client.models.generate_content(
            model=project_model, config=config_fncall, contents=contents
        )
        # A part can be a function call or natural language response.
        for part in response.candidates[0].content.parts:
            if function_call := part.function_call:
                # Extract the function call.
                fn_name = function_call.name
                #display(Markdown("#### Predicted function name"))
                #print(fn_name, "\n")
                # Extract the function call arguments.
                fn_args = {key: value for key, value in function_call.args.items()}
                #display(Markdown("#### Predicted function arguments"))
                #print(fn_args, "\n")
                # Call the predicted function.
                api_response = function_handler[fn_name](fn_args)[:20000] # Stay within the input token limit
                #display(Markdown("#### API response"))
                #print(api_response[:500], "...", "\n")
                # Create an API response part.
                api_response_part = types.Part.from_function_response(
                    name=fn_name,
                    response={"content": api_response},
                )
                # Append the model's function call part.
                contents.append(types.Content(role="model", parts=[types.Part(function_call=function_call)])) 
                # Append the api response part.
                contents.append(types.Content(role="user", parts=[api_response_part]))
            else:
                # The model gave a natural language response
                function_calling_in_process = False
                break # No more parts in response.
        if not function_calling_in_process:
            break # The function calling chain is complete.
            
    # Show the final natural language summary
    display(Markdown("#### Natural language response"))
    display(Markdown(response.text.replace("$", "\\\\$")))

# Ask a question

In [22]:
send_message("What is the current price of Amazon stock?")

Generate quote embedding: 0it [00:00, ?it/s]


#### Natural language response

The current price of Amazon (AMZN) stock is \\$189.98. The price changed by -\\$0.22, which is -0.1157% since the previous close. The high price of the day was \\$192.88, and the low was \\$186.4. The opening price for the day was \\$191.435, and the previous close price was \\$190.2. The price was last updated on Saturday, May 3, 2025 at 4:00:00 PM.


In [23]:
send_message(
    """Tell me Amazon's current share price and provide candlestick data for the past month.
    Sort the data in descending order by date. Format the prices consistently as currency.
    Round prices to two decimal places.
    Present the data with multiple columns for display in markdown.""")

Generate api embedding: 0it [00:00, ?it/s]


#### Natural language response

As of May 2, 2025, at midnight, the current price of Amazon (AMZN) is \\$189.98.

Here is the candlestick data for the past month, sorted in descending order by date:

| Date             | Open    | High    | Low     | Close   | Volume      |
| ---------------- | ------- | ------- | ------- | ------- | ----------- |
| Fri May 2 2025   | \\$191.44 | \\$192.88 | \\$186.40 | \\$189.98 | 77,677,487  |
| Thu May 1 2025   | \\$190.63 | \\$191.81 | \\$187.50 | \\$190.20 | 74,228,963  |
| Wed Apr 30 2025  | \\$182.17 | \\$185.05 | \\$178.85 | \\$184.42 | 55,176,543  |
| Tue Apr 29 2025  | \\$183.99 | \\$188.02 | \\$183.68 | \\$187.39 | 41,667,255  |
| Mon Apr 28 2025  | \\$190.11 | \\$190.22 | \\$184.89 | \\$187.70 | 33,224,732  |
| Fri Apr 25 2025  | \\$187.62 | \\$189.94 | \\$185.49 | \\$188.99 | 36,413,330  |
| Thu Apr 24 2025  | \\$180.92 | \\$186.74 | \\$180.18 | \\$186.54 | 43,051,696  |
| Wed Apr 23 2025  | \\$183.45 | \\$187.38 | \\$180.19 | \\$180.60 | 63,470,094  |
| Tue Apr 22 2025  | \\$169.85 | \\$176.78 | \\$169.35 | \\$173.18 | 56,607,202  |
| Mon Apr 21 2025  | \\$169.60 | \\$169.60 | \\$165.29 | \\$167.32 | 48,126,111  |
| Thu Apr 17 2025  | \\$176.00 | \\$176.21 | \\$172.00 | \\$172.61 | 44,726,453  |
| Wed Apr 16 2025  | \\$176.29 | \\$179.10 | \\$171.41 | \\$174.33 | 51,866,916  |
| Tue Apr 15 2025  | \\$181.41 | \\$182.35 | \\$177.93 | \\$179.59 | 43,617,902  |
| Mon Apr 14 2025  | \\$186.84 | \\$187.44 | \\$179.23 | \\$182.12 | 48,002,540  |
| Fri Apr 11 2025  | \\$179.93 | \\$185.86 | \\$178.00 | \\$184.87 | 50,594,339  |
| Thu Apr 10 2025  | \\$185.44 | \\$186.87 | \\$175.85 | \\$181.22 | 68,302,045  |
| Wed Apr 9 2025   | \\$172.12 | \\$192.65 | \\$169.93 | \\$191.10 | 116,804,328 |
| Tue Apr 8 2025   | \\$185.23 | \\$185.90 | \\$168.57 | \\$170.66 | 87,710,360  |
| Mon Apr 7 2025   | \\$162.00 | \\$183.41 | \\$161.38 | \\$175.26 | 109,297,115 |
| Fri Apr 4 2025   | \\$167.15 | \\$178.14 | \\$166.00 | \\$171.00 | 123,136,859 |

The share price has fluctuated over the past month, with a general upward trend. The volume of shares traded has also varied.


In [24]:
send_message(
    '''Tell me about Amazon's current bullish versus bearish predictions, and recommendation trends.
    Include a discussion of any short-term trends, and sentiment analysis.''')

Generate api embedding: 0it [00:00, ?it/s]


#### Natural language response

As of May 4, 2025, here's a summary of the bullish versus bearish predictions and recommendation trends for Amazon, along with sentiment analysis and short-term trends:

**Recommendation Trends:**

*   Based on the latest analyst recommendation trends, there is a strong bullish sentiment towards Amazon.
*   In May 2025, there were 22 strong buy, 51 buy, and 6 hold recommendations. There were no sell or strong sell recommendations.
*   The trend has remained consistently positive over the past few months, with a high number of buy and strong buy recommendations.

**Sentiment Analysis:**

*   Recent news articles present a mixed sentiment regarding Amazon.
*   Several articles highlight the potential negative impact of tariffs on Amazon's e-commerce business. Concerns exist about increased prices for consumers and potential disruptions to the supply chain.
*   However, many articles emphasize Amazon's strengths, including its dominant position in e-commerce, the growth potential of Amazon Web Services (AWS), and investments in artificial intelligence (AI).
*   Some analysts believe that Amazon is well-positioned to weather the tariff storm and may even benefit in the long run.
*   Recent news suggests that Amazon has debunked rumors that have hurt AI-related stocks.
*   Some articles suggest that Amazon is a good long-term investment due to its diversified revenue streams and growth opportunities in AI and logistics.
*   There are also articles noting that Amazon is underperforming the S\&P 500 so far in 2025.

**Short-Term Trends:**

*   The stock market may experience volatility due to key earnings reports from major tech companies like Amazon, as well as economic data releases.
*   There are concerns about the impact of tariffs and a potential economic slowdown on Amazon's business.
*   However, Amazon's cloud computing business (AWS) and advertising segments are expected to show growth.
*   Some analysts believe that Amazon's earnings report could accelerate the stock's recent bounce if it meets or exceeds expectations.

**Overall Summary:**

While there are some concerns about the short-term impact of tariffs and economic uncertainty, the overall sentiment towards Amazon remains positive. Analysts are generally bullish on the stock, citing its strong competitive advantages, diversified revenue streams, and growth opportunities in cloud computing and AI.


In [25]:
send_message("What is Google's stock ticker symbol?")

Score similarity to query: 100%|██████████| 11/11 [00:00<00:00, 16.45it/s]
Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Generate wiki embeddings: 0it [00:00, ?it/s]


#### Natural language response

Google's stock ticker symbols on the NASDAQ are GOOGL and GOOG. It is also listed on the Frankfurt Stock Exchange under the ticker symbol GGQ1. These symbols refer to Alphabet Inc., Google's holding company.


In [27]:
send_message(
    '''Tell me about Google's share price over the past month.
    Perform a sentiment analysis of news during the same period. Include trends.''')

Score similarity to query: 100%|██████████| 11/11 [00:02<00:00,  4.52it/s]
Generate api embedding: 0it [00:00, ?it/s]


#### Natural language response

Here's a summary of Google's (Alphabet Inc.) share price and recent news sentiment over the past month (April 3, 2025, to May 3, 2025):

**Share Price Summary:**

The share price of Alphabet Inc. (GOOGL) has fluctuated over the past month.

*   April 3, 2025: The closing price was \\$150.72.
*   April 4, 2025: The closing price decreased to \\$145.60.
*   April 7, 2025: The closing price increased to \\$146.75.
*   April 8, 2025: The closing price decreased to \\$144.70.
*   April 9, 2025: The closing price increased significantly to \\$158.71.
*   April 10, 2025: The closing price decreased to \\$152.82.
*   April 11, 2025: The closing price increased to \\$157.14.
*   April 14, 2025: The closing price increased to \\$159.07.
*   April 15, 2025: The closing price decreased to \\$156.31.
*   April 16, 2025: The closing price decreased to \\$153.33.
*   April 17, 2025: The closing price decreased to \\$151.16.
*   April 21, 2025: The closing price decreased to \\$147.67.
*   April 22, 2025: The closing price increased to \\$151.47.
*   April 23, 2025: The closing price increased to \\$155.35.
*   April 24, 2025: The closing price increased to \\$159.28.
*   April 25, 2025: The closing price increased to \\$161.96.
*   April 28, 2025: The closing price decreased to \\$160.61.
*   April 29, 2025: The closing price decreased to \\$160.16.
*   April 30, 2025: The closing price decreased to \\$158.80.
*   May 1, 2025: The closing price increased to \\$161.30.
*   May 2, 2025: The closing price increased to \\$164.03.

**News Sentiment Analysis:**

The news sentiment surrounding Alphabet has been mixed over the past month. Here's a breakdown of the key themes and sentiments:

*   **Artificial Intelligence (AI):** AI remains a dominant theme, with many articles discussing Alphabet's AI investments, innovations (like Gemini), and the potential impact of AI on its various businesses. Sentiment is generally positive, with analysts highlighting Alphabet's strong position in the AI race and its potential to benefit from the growing AI market.
*   **Financial Performance:** Several articles discuss Alphabet's strong financial performance, including revenue growth, expanding margins, and a \\$70 billion share repurchase program. This has generally led to positive sentiment, with analysts viewing the stock as undervalued.
*   **Antitrust Concerns:** Antitrust lawsuits and regulatory scrutiny continue to be a concern, with some articles highlighting the potential risks to Alphabet's search and advertising businesses. This has contributed to some negative sentiment.
*   **Tariffs and Trade Wars:** Concerns about the impact of tariffs and trade wars on the U.S. economy and Alphabet's business have also weighed on sentiment.
*   **Analyst Ratings:** Analyst ratings have been mixed, with some analysts lowering price targets while others remain bullish on the company's long-term potential.
*   **Partnerships and Collaborations:** Alphabet's partnerships and collaborations with other companies, such as Palantir and Clover Health, have generally been viewed positively.

**Trends:**

*   AI continues to be a major focus for Alphabet, with the company investing heavily in AI infrastructure and developing new AI solutions.
*   The company's financial performance remains strong, but there are concerns about the potential impact of antitrust lawsuits and trade wars.
*   Analysts are divided on Alphabet's near-term prospects, but many remain bullish on the company's long-term potential.

**Concise Summary:**

Alphabet's share price has experienced volatility over the past month, influenced by factors such as AI developments, financial performance, antitrust concerns, and macroeconomic uncertainties. News sentiment has been mixed, with positive coverage of Alphabet's AI leadership and financial strength offset by concerns about regulatory scrutiny and trade tensions. While the near-term outlook is uncertain, analysts generally remain optimistic about Alphabet's long-term growth potential.


In [28]:
send_message("What is MGM Studio's stock symbol?")

Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Generate wiki embeddings: 0it [00:00, ?it/s]


#### Natural language response

I am unable to find the stock symbol for Mgm Studios.

In [29]:
send_message("What is Amazon MGM Studio's stock symbol?")

Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Generate wiki embeddings: 0it [00:00, ?it/s]
Generate grounding embedding: 0it [00:00, ?it/s]


#### Natural language response

Amazon MGM Studios is a subsidiary of Amazon, which has the stock symbol AMZN.


In [30]:
send_message("What is Facebook's stock ticker symbol?")

Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Generate wiki embeddings: 0it [00:00, ?it/s]
Generate grounding embedding: 0it [00:00, ?it/s]


#### Natural language response

The stock ticker symbol for Facebook, now known as Meta Platforms Incorporated, is META. It is traded on the NASDAQ. Facebook's initial public offering was on May 18, 2012, under the ticker symbol FB, but the ticker symbol was changed to META on June 9.


In [31]:
send_message(
    '''How is the outlook for Apple based on trends and news sentiment over the past month?
    Perform the same analysis on Apple's peers. Then compare Apple result to it's peers.''')

Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 18/18 [00:00<00:00, 81.20it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 18/18 [00:00<00:00, 80.31it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 18/18 [00:00<00:00, 82.32it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 18/18 [00:00<00:00, 79.25it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 17/17 [00:00<00:00, 75.43it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 18/18 [00:00<00:00, 81.39it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 18/18 [00:00<00:00, 80.57it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 17/17 [00:00<00:00, 77.80it/s]
Generate api embedding: 0it [00:00, ?it/s]
Generate peer

#### Natural language response

Based on the available data, here's a concise and detailed summary of the outlook for Apple and its peers:

**Apple (AAPL):**

*   **Analyst Recommendations:** Analyst recommendations for Apple show a generally positive sentiment, with a mix of "strong buy," "buy," and "hold" ratings over the past few months. There are very few "sell" or "strong sell" recommendations.
*   **News Sentiment:** The news sentiment surrounding Apple is mixed. Some articles point to an uncertain outlook due to tariff concerns and weaker sales in China. Other articles highlight Apple's strong Q2 results, cash generation, and steps to mitigate tariff impacts. There are also concerns about regulatory challenges and potential headwinds for the company.
*   **Financial Indicators:** Apple has a current dividend yield of 0.4964%. The price to earnings ratio is 31.7058.

**Peer Analysis:**

*   **Peers:** Apple's peers based on subIndustry are Dell Technologies, HP Inc, Hewlett Packard Enterprise, Super Micro Computer Inc, NetApp Inc, Pure Storage Inc, Western Digital Corp, and IonQ Inc.
*   **Dell Technologies (DELL):** News sentiment is generally positive, with articles highlighting its AI capabilities, strong earnings, and potential for growth in the AI server market.
*   **HP Inc. (HPQ):** News sentiment is mixed, with some articles highlighting positive collaborations and initiatives, while others point to concerns about earnings growth and potential tariff impacts.
*   **Super Micro Computer Inc (SMCI):** News sentiment is highly volatile. While the company is a key player in the AI server market, it has faced accounting issues, regulatory probes, and concerns about its financial reliability.
*   **NetApp Inc (NTAP):** No news sentiment could be found.
*   **Pure Storage Inc (PSTG):** No news sentiment could be found.
*   **Western Digital Corp (WDC):** No news sentiment could be found.
*   **IonQ Inc (IONQ):** No news sentiment could be found.

**Comparison:**

*   Apple faces some headwinds related to tariffs and sales in China, but it maintains a strong financial position and is taking steps to mitigate these challenges.
*   Apple's peers, particularly Dell Technologies and Hewlett Packard Enterprise, are also benefiting from the growing demand for AI infrastructure and cloud computing. However, some of these companies face their own challenges, such as accounting issues or declining PC sales.
*   Super Micro Computer, while a key player in the AI server market, faces significant risks related to its financial reporting and regulatory concerns.

**Overall:**

The outlook for Apple is cautiously optimistic. While the company faces some challenges, it remains a strong player in the tech industry with a solid financial foundation. The outlook for Apple's peers is more varied, with some companies showing strong growth potential and others facing significant risks.


In [34]:
send_message('''What does the recent news say about Apple and the impact of tariffs? Over the past 2 month.''')

#### Natural language response

Based on recent news, here's a summary of the impact of tariffs on Apple over the past two months:

*   **Tariff Impact on Earnings:** Apple's fiscal Quarter 2 earnings were impacted by tariffs, and the company anticipates a \\$900 million increase in tariff-related costs in Quarter 3.
*   **Mitigation Efforts:** Apple has been taking steps to mitigate the effects of tariffs by building up inventory and shifting production to countries like India and Vietnam to avoid higher tariffs on Chinese imports.
*   **Uncertain Outlook:** Despite better-than-expected Quarter 2 results, Apple faces an uncertain outlook due to tariff concerns and weaker sales in China.
*   **Headwinds:** Apple faces multiple challenges, including tariff-related cost increases and supply chain issues, leading to concerns about its near-term growth prospects.
*   **Analyst Downgrade:** Bank of America cut Apple's price target due to tariff risks and a delayed artificial intelligence rollout for future iPhones.
*   **Temporary Relief:** Apple received a temporary exemption from tariffs on its electronics, providing some relief.
*   **Long-Term Concerns:** While tariffs are a short-term concern, the larger issue is the declining earnings quality of major companies like Apple.
*   **China Sales:** Weaker iPhone sales in China have caused Apple to lag in the market.
*   **Production Shifts:** Apple is taking steps to mitigate the tariff impact by shifting production to India and Vietnam.
*   **Tariff Exemptions:** Apple has received a temporary exemption from tariffs on its electronics.
*   **Analyst View:** A Wall Street analyst sees upside potential for the stock, citing Apple's strong market share in the premium hardware sector and its flywheel model.
*   **Geopolitical Tensions:** Apple is at the center of increasing geopolitical tensions between the United States and China.
*   **Tariff Risks:** Bank of America cut Apple's price target due to tariff risks and delayed artificial intelligence rollout for future iPhones. Short-term sales benefit from tariff-driven demand pull-forward, but supply chain costs threaten 2026 margins.
*   **Tariff Turbulence:** Apple is heavily reliant on manufacturing in China, but the article notes that it has expanded to other countries and the electronics industry is currently exempt from tariffs, at least temporarily.
*   **Tariff Pause:** Apple stock surged after the Trump administration temporarily paused reciprocal tariffs on smartphones, computers, and other electronics. However, uncertainty remains as Apple is still subject to the initial 20% tariffs on China.
*   **Tariff Relief:** Apple has received a temporary exemption from tariffs on its electronics.
*   **Tariff Impact:** Apple is at the center of increasing geopolitical tensions between the United States and China, and investors will want to know how the business is coping.
*   **Tariff Concerns:** Apple's quarterly earnings fell short of expectations, with concerns over tariffs and underperforming divisions.
*   **Tariff Worries:** Apple's upcoming earnings report faces uncertainty as investors brace for potential guidance cuts amid ongoing United States-China trade tensions, despite expectations of another earnings beat.
*   **Tariff Challenges:** Apple faces challenges in China due to trade tensions and the shift of iPhone production out of the country.
*   **Tariff Pressures:** Apple is facing rising cost pressures.
*   **Tariff Sensitive:** Apple is a tariff-sensitive stock.
*   **Tariff Unstoppable?:** Apple stock plunged due to tariffs on goods from China.
*   **Tariff Relief:** Three tech stocks, Apple, Hewlett-Packard, and Nvidia, have rebounded due to tariff relief.
*   **Tariff Impact:** Apple's earnings were impacted by tariffs, but the company has taken steps to mitigate the effects, such as building up inventory and shifting production to other countries. The overall impact was limited in the latest quarter.
*   **Tariff Mitigation:** Apple is taking steps to mitigate the tariff impact by shifting production to India and Vietnam.
*   **Tariff Concerns:** Apple beat on earnings and revenue, but faces an uncertain outlook due to tariff concerns and weaker sales in China. The company is taking steps to mitigate the tariff impact, but the future impact remains unclear.
*   **Tariff Headwinds:** Apple is facing significant headwinds, including a court ruling that undermines its App Store revenue model, tariff-related cost increases, and supply chain challenges, which are negatively impacting its stock performance and growth outlook.
*   **Tariff Impact:** Apple's earnings were impacted by tariffs, but the company has taken steps to mitigate the effects, such as building up inventory and shifting production to other countries. The overall impact was limited in the latest quarter.
*   **Tariff Concerns:** Apple beat on earnings and revenue, but faces an uncertain outlook due to tariff concerns and weaker sales in China. The company is taking steps to mitigate the tariff impact, but the future impact remains unclear.
*   **Tariff Headwinds:** Apple is facing significant headwinds, including a court ruling that undermines its App Store revenue model, tariff-related cost increases, and supply chain challenges, which are negatively impacting its stock performance and growth outlook.
*   **Tariff Impact:** Apple's earnings were impacted by tariffs, but the company has taken steps to mitigate the effects, such as building up inventory and shifting production to other countries. The overall impact was limited in the latest quarter.
*   **Tariff Concerns:** Apple beat on earnings and revenue, but faces an uncertain outlook due to tariff concerns and weaker sales in China. The company is taking steps to mitigate the tariff impact, but the future impact remains unclear.
*   **Tariff Headwinds:** Apple is facing significant headwinds, including a court ruling that undermines its App Store revenue model, tariff-related cost increases, and supply chain challenges, which are negatively impacting its stock performance and growth outlook.

In summary, while Apple has taken steps to mitigate the impact of tariffs, they continue to pose a risk to the company's earnings and future growth.


# Conclusion

<span style="font-size:18px;">
For now that will have to do. Our Essy has a solid foundation but more could be done to organise metadata. No evaluation or validation has been performed (except fuzzing the prompt). Next steps include restructuring the vector database based on lessons learned. That'll be followed by plotting, multi-modal, and structured output. The last close date (generative) function can be temperamental. In the same way Gemini always feels regarding dates. I've learnt so much. I'm happy I decided to participate in the event! It really has been a joy to see Essy grow from random chat with Gemini into the foundation for a good-broker buddy. I hope you enjoy playing with this edition as much as I enjoyed building it!
</span>